In [4]:
from google.colab import drive
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import word_tokenize
from sentence_transformers import SentenceTransformer, util
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
from textblob import TextBlob

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
files_path = '/content/drive/MyDrive/QBC AI/Project_2/'
train_df = pd.read_csv(files_path + "train_data.csv")
test_df = pd.read_csv(files_path + "test_data.csv")
title_brand_df = pd.read_csv(files_path + "title_brand.csv")

<ipython-input-6-643915f0df6e>:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(files_path + "train_data.csv")


<div dir="rtl">

### مشخصات داده‌ها:

- **overall**: امتیاز محصول (توسط فرد نظر دهنده) از ۱ تا ۵  
- **vote**: تعداد رای‌های دیدگاه از نظر مفید بودن (helpful)  
- **verified**: آیا تأیید و منتشر شده است یا خیر  
- **reviewTime**: تاریخ ثبت نظر  
- **reviewerID**: شناسه‌ی شخص نظر دهنده  
- **Asin**: شناسه‌ی محصول (برای دسترسی به لینک محصول می‌توانید شناسه را بعد از `https://www.amazon.com/dp/` قرار دهید)  
- **style**: دیکشنری شامل برخی توضیحات محصول مثل رنگ، سایز و غیره  
- **reviewerName**: نام شخص نظر دهنده  
- **reviewText**: متن نظر  
- **summary**: خلاصه‌ی نظر  
- **unixReviewTime**: زمان ثبت نظر با فرمت Unix Time  

</div>


In [ ]:
train_df.sort_values('reviewTime')

,overall,vote,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime
400540,5,NaN,False,2016-01-01,A38LIG44819X1N,B00T63YVAM,{'Size:': ' 70-Inch'},Jesse CreateThis,I just bought one of these through my local BB...,Wow. Just. Wow. This thing is awesome.,1451606400
20192,5,18.0,True,2016-01-01,A12034ZBKBKK0A,B000ESCAUQ,NaN,J. Alfred,A great option for any 7 foot parabolic umbrel...,Great addition to a great modifier...,1451606400
20188,5,2.0,True,2016-01-01,A10AQMEFDTT7J5,B000ERAIW4,NaN,Paula Johnson,My 4-year old son loves his new Educational In...,Nice binoculars for kids!,1451606400
589772,5,NaN,False,2016-01-01,AZA595ZPIG240,B0185C55TK,{'Color:': ' Black'},Parker Knight,Case is made of durable plastic with inside de...,Provides superior impact protection and spread...,1451606400
159589,4,NaN,False,2016-01-01,A2R7V8V1WIQH9K,B00B0CQCCC,NaN,dreamworlds,I purchased this router from other online reta...,"if you have problem on 5GHz network, read this.",1451606400
...,...,...,...,...,...,...,...,...,...,...,...
837699,5,NaN,True,2018-10-02,A2ZE26LV7HVSAD,B01GUITK24,{'Style Name:': ' Bookshelf Speaker Mount 2 Pa...,Big Richard,I had to use my own drywall anchors which were...,Solid and kind of heavy,1538438400
838669,2,NaN,True,2018-10-02,A22TE5RFEO9KV9,B01HETFQA8,{'Color:': ' Black'},BH,I ordered my QC30 headphones 10 months ago. Th...,Really Nice While They Lasted,1538438400
837901,2,NaN,True,2018-10-02,A2U26O5EYD1RB4,B01GZRIJRC,NaN,Random Reviewer,Bought this screen protector for my D850. The...,Glass peels up from the corner after short time,1538438400
838766,5,NaN,True,2018-10-03,A32FN9RNY65CKH,B01HGF6XTI,{'Size:': ' Charger'},RicM,Went to Boston and forgot my charger.\nOrdered...,Worked fine - got me through in a pinch,1538524800


In [ ]:
title_brand_df

,asin,title,brand
0,0011300000,Genuine Geovision 1 Channel 3rd Party NVR IP S...,GeoVision
1,0043396828,"Books ""Handbook of Astronomical Image Processi...",33 Books Co.
2,0060009810,One Hot Summer,Visit Amazon's Carolina Garcia Aguilera Page
3,0060219602,Hurray for Hattie Rabbit: Story and pictures (...,Visit Amazon's Dick Gackenbach Page
4,0060786817,sex.lies.murder.fame.: A Novel,Visit Amazon's Lolita Files Page
...,...,...,...
786440,B01HJGOOMW,Tukzer Cassette Player Portable Tape Player Ca...,Tukzer
786441,B01HJH3XLE,Cobiter AC Adapter Battery Charger For IBM LEN...,Cobiter
786442,B01HJH6CEY,Baofeng Original Li-ion Battery Charger Deskto...,COOLBOY
786443,B01HJH40WU,Jricoo 3pack 6ft Micro USB to USB Cable 2.0 6f...,Jricoo


In [7]:
# ساخت دیتافریم با ستونهای مورد نیاز
guaranty_df = train_df[['overall', 'reviewText','asin']]

In [8]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# لیست کلمات کلیدی مرتبط
words = [
    "insurance", "assurance", "guaranty", "policy", "coverage", "refund", "repair",
    "return", "compensation", "warrantee", "service", "replacement", "guaranteed",
    "protection", "commitment", "contract", "support", "trust", "responsibility",
    "obligation", "guaranteeing", "assurance", "pledge", "promise", "security",
    "certainty", "commitment", "bond", "vow", "oath",
    "protection", "backing", "coverage", "safeguard", "endorsement", "surety",
    "affirmation", "covenant", "agreement", "guarantee", "certification",
    "reassurance", "undertaking", "safeguard", "confirmation", "indemnity",
    "coverage", "liability", "guaranteeing", "responsibility", "warranty agreement",
    "assurance contract", "reaffirmation", "vouching", "validation", "seal",
    "guarantee period", "surety bond", "security bond", "safeguarding", "protection plan",
    "risk management", "damage control", "trust", "backup", "certificate",
    "promise of repair", "product protection", "warranty"
]

# کلمات هدف
target_words = ["warranty", "guarantee"]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# دریافت بردارهای معنایی برای کلمات هدف
warranty_embedding = model.encode("warranty", convert_to_tensor=True)
guarantee_embedding = model.encode("guarantee", convert_to_tensor=True)

# دریافت بردارهای معنایی برای کلمات مرتبط
word_embeddings = model.encode(words, convert_to_tensor=True)

# محاسبه شباهت کسینوسی کلمات "warranty" و "guarantee" با کلمات دیگر
similarities_warranty = util.pytorch_cos_sim(warranty_embedding, word_embeddings)
similarities_guarantee = util.pytorch_cos_sim(guarantee_embedding, word_embeddings)

# ترکیب دو لیست شباهت
similarities = (similarities_warranty + similarities_guarantee) / 2

# مرتب‌سازی بر اساس بالاترین شباهت
sorted_indices = torch.argsort(similarities, descending=True)

top_n = 20
similar_words = [words[i] for i in sorted_indices[0][:top_n]]
print("کلمات مشابه:", similar_words)

کلمات مشابه: ['warranty', 'guarantee', 'warrantee', 'guaranteeing', 'guaranteeing', 'warranty agreement', 'guaranteed', 'guarantee period', 'promise of repair', 'assurance', 'assurance', 'repair', 'insurance', 'refund', 'trust', 'trust', 'certainty', 'policy', 'obligation', 'assurance contract']


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

all_words =  words

# ایجاد CountVectorizer برای تبدیل کلمات به ویژگی‌ها
vectorizer = CountVectorizer().fit(all_words)

# تبدیل کلمات هدف و کلمات مشابه به ویژگی‌های BoW
word_vectors = vectorizer.transform(all_words)

# محاسبه شباهت کسینوسی
similarity_matrix = cosine_similarity(word_vectors)

# شباهت کلمات "warranty" و "guarantee" با کلمات مشابه
warranty_idx = all_words.index("warranty")
guarantee_idx = all_words.index("guarantee")

# شباهت کسینوسی کلمات هدف با دیگر کلمات
similarities_warranty = similarity_matrix[warranty_idx]
similarities_guarantee = similarity_matrix[guarantee_idx]

# ترکیب شباهت‌ها (میانه یا میانگین)
combined_similarities = (similarities_warranty + similarities_guarantee) / 2

# مرتب‌سازی شباهت‌ها از بیشترین به کمترین
sorted_indices = combined_similarities.argsort()[::-1]

# انتخاب کلمات مشابه برتر
top_n = 20
similar_words_bow = [all_words[i] for i in sorted_indices[:top_n]]
print("کلمات مشابه با BoW:", similar_words_bow)


کلمات مشابه با BoW: ['warranty', 'guarantee', 'guarantee period', 'warranty agreement', 'policy', 'coverage', 'protection', 'oath', 'vow', 'bond', 'commitment', 'assurance', 'certainty', 'security', 'promise', 'pledge', 'assurance', 'guaranteeing', 'obligation', 'responsibility']


In [ ]:
print(similar_words_bow)
print(similar_words)

['warranty', 'guarantee', 'guarantee period', 'warranty agreement', 'policy', 'coverage', 'protection', 'oath', 'vow', 'bond', 'commitment', 'assurance', 'certainty', 'security', 'promise', 'pledge', 'assurance', 'guaranteeing', 'obligation', 'responsibility']
['warranty', 'guarantee', 'warrantee', 'guaranteeing', 'guaranteeing', 'warranty agreement', 'guaranteed', 'guarantee period', 'promise of repair', 'assurance', 'assurance', 'repair', 'insurance', 'refund', 'trust', 'trust', 'certainty', 'policy', 'obligation', 'assurance contract']


In [11]:
# پیدا کردن کلمات در similar_words که در similar_words_bow نیستند
unique_to_words = list(set(similar_words) - set(similar_words_bow))

# پیدا کردن کلمات در similar_words_bow که در similar_words نیستند
unique_to_words_bow = list(set(similar_words_bow) - set(similar_words))

# نمایش نتایج
print("کلمات منحصر به فرد در similar_words:", unique_to_words)
print("کلمات منحصر به فرد در similar_words_bow:", unique_to_words_bow)

کلمات منحصر به فرد در similar_words: ['assurance contract', 'promise of repair', 'repair', 'warrantee', 'guaranteed', 'refund', 'insurance', 'trust']
کلمات منحصر به فرد در similar_words_bow: ['bond', 'security', 'coverage', 'pledge', 'promise', 'commitment', 'responsibility', 'vow', 'oath', 'protection']


In [12]:
combined_words = list(set(similar_words) | set(similar_words_bow))

# به‌طور پیش‌فرض ترتیب کلمات را در یک لیست قرار می‌دهیم (بر اساس اولین قرارگیری در مجموعه‌ها)
combined_words_sorted = sorted(combined_words, key=lambda x: (similar_words + similar_words_bow).index(x))

# چاپ اولین 25 کلمه
top_25_words = combined_words_sorted[:25]
print("25 کلمه برتر اجتماع دو مجموعه:", top_25_words)

25 کلمه برتر اجتماع دو مجموعه: ['warranty', 'guarantee', 'warrantee', 'guaranteeing', 'warranty agreement', 'guaranteed', 'guarantee period', 'promise of repair', 'assurance', 'repair', 'insurance', 'refund', 'trust', 'certainty', 'policy', 'obligation', 'assurance contract', 'coverage', 'protection', 'oath', 'vow', 'bond', 'commitment', 'security', 'promise']


In [14]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')
nltk.download('punkt_tab')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [15]:
from nltk.tokenize import word_tokenize

def tokenize_reviews(review_text):
    # توکن‌سازی نظرات
    return word_tokenize(review_text)

# اعمال توکن‌سازی بر روی نظرات
guaranty_df['tokenized_reviews'] = guaranty_df['reviewText'].apply(tokenize_reviews)


<ipython-input-15-ff1421e47201>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  guaranty_df['tokenized_reviews'] = guaranty_df['reviewText'].apply(tokenize_reviews)


In [17]:
def correct_spelling(text):
    blob = TextBlob(text)
    return str(blob.correct())


In [18]:
filter_words = top_25_words

# تصحیح غلط‌های املایی در نظرات
#guaranty_df['reviewText'] = guaranty_df['reviewText'].apply(correct_spelling)

# فیلتر کردن نظرات که شامل کلمات مورد نظر باشند
filtered_reviews = guaranty_df[guaranty_df["reviewText"].str.contains('|'.join(filter_words), case=False, na=False)]

# محاسبه میانگین امتیاز برای هر محصول
avg_scores = filtered_reviews.groupby("asin")["overall"].mean().reset_index()
avg_scores.columns = ["Product ID", "Average Rating on Warranty"]

In [ ]:
filter_words_set = set(top_25_words)

def contains_target_words(tokenized_review, filter_words_set):
    return any(word.lower() in filter_words_set for word in tokenized_review)

# Apply the filtering function on the tokenized reviews
filtered_reviews = guaranty_df[guaranty_df['tokenized_reviews'].apply(lambda x: contains_target_words(x, filter_words_set))]

# Now calculate the average score for each product (asin)
avg_scores = filtered_reviews.groupby("asin")["overall"].mean().reset_index()
avg_scores.columns = ["Product ID", "Average Rating on Warranty"]


In [ ]:
avg_scores

,Product ID,Average Rating on Warranty
0,0511189877,4.000000
1,106171327X,5.000000
2,140053271X,4.000000
3,140069938X,4.000000
4,1495443043,3.000000
...,...,...
25705,B01HIWPMNW,4.500000
25706,B01HIZEW1C,4.500000
25707,B01HJ8E11E,5.000000
25708,B01HJDNL60,3.333333


In [ ]:
train_df[train_df['asin'] == 'B01HJDNL60'	]['reviewText']

,reviewText
734150,"I reviewed the R2 Camera, so naturally being i..."
734151,Camera works great. Mount takes some work to s...
734152,I would not purchase this dashcam. It's lasted...
734153,Failed when we need it today. The cam was stu...
734154,Update 1/26/18\n\nUpdating my rating to 1 star...
734155,My initial impressions of this dashcam are goo...
734156,This little camera catches excellent video and...
734157,Great image and easy to install. Could have be...
734158,This is a nice compact camera. I am very impre...
734159,"I wish I can give this 5 stars, but a few week..."


In [ ]:
filtered_reviews[filtered_reviews['asin'] == 'B01HJDNL60'	][['reviewText', 'overall']]

,reviewText,overall
734155,My initial impressions of this dashcam are goo...,4
734158,This is a nice compact camera. I am very impre...,3
838937,This review covers the Vantrue N1 and X2 dash ...,3


In [ ]:
print(train_df.loc[838937, 'reviewText'])

This review covers the Vantrue N1 and X2 dash cameras. I purchased both cameras and the optional GPS-equipped mounts. I have had no trouble with the N1 camera itself. The X2 was very problematic and I thought I'd never get it going. The camera griped about the formatting of the SD card (a high-quality class 10 card), even when the card was formatted by the camera. It froze, refused to turn off, emitted a loud hissing sound, etc. Contrary to what I've read here, Vantrue customer support was excellent. They replaced the camera with a new one, which worked just fine.

As others have said, the mounts have major problems sticking to the windshield. This includes the GPS-equipped mounts. Once again Vantrue's support folks came through and sent a new replacement mount, but it had the same problem staying on the glass. They finally offered to take everything back for a refund or to pay for a 3M sticky pad to attach the mount. I like the camera so I chose that route and I have had no further pr

#vector kardm k moshkelate contains ro nadashte bashe: promise in compromises
# corection -> correction
# typo -> type
# nemudar
# tarkib boW and Word Embedding

In [ ]:
text = """
I wish I can give this 5 stars, but a few weeks after I purchased it, they suddenly went on sale for $25 dollars less and it's pissing me off.

As for the product itself, the product feel high quality, hard textured plastic or actual steel I'm not sure.  Software is cleaner than your average Chinese made and bought dashcams.  power connector from the mount is a great idea, but unfortunately it compromises the stiffness of the joint connecting the camera with the mount; the camera will move forward if you stop hard enough, imagine it with a frontal collision.

Night shots are really not that great, drive through a highway lit up by LED lamp posts and you can only see things in front of you that are directly lit up by your headlight.  At least you can see the plate of the car in front.

Inserting the SDcard was a nightmare, your fingernails are not enough, had to use a butterknife to push it in.

So yeah, it's ok, but after the price slash I really regretted buying this.
"""

# جستجو برای کلمات موجود در متن
found_words = [word for word in words if word.lower() in text.lower()]

print("کلمات یافت‌شده:", found_words)

کلمات یافت‌شده: ['promise']
